In [7]:
import os, itertools, random, argparse, time, datetime,torch
## task type = 0
ap = argparse.ArgumentParser()
ap.add_argument('--tasktype', type=int, default='0', help="task type, 0: dynamics,1:node_level, 2:edge_level")
ap.add_argument('--dataset', type=str, default='region785', help="Dataset string")
ap.add_argument('--n_layer', type=int, default=2, help="number of layers (default 1)")
ap.add_argument('--n_hidden', type=int, default=20, help="rnn hidden states (could be set as any value)")
ap.add_argument('--seed', type=int, default=42, help='random seed')
ap.add_argument('--epochs', type=int, default=15, help='number of epochs to train')
ap.add_argument('--lr', type=float, default=1e-3, help='initial learning rate')
ap.add_argument('--weight_decay', type=float, default=5e-4, help='weight decay (L2 loss on parameters).')
ap.add_argument('--dropout', type=float, default=0.2, help='dropout rate usually 0.2-0.5.')
ap.add_argument('--batch', type=int, default=128, help="batch size")
ap.add_argument('--check_point', type=int, default=1, help="check point")
ap.add_argument('--shuffle', action='store_true', default=False, help="not used, default false")
ap.add_argument('--train', type=float, default=.5, help="Training ratio (0, 1)")
ap.add_argument('--val', type=float, default=.2, help="Validation ratio (0, 1)")
ap.add_argument('--test', type=float, default=.3, help="Testing ratio (0, 1)")
ap.add_argument('--mylog', action='store_false', default=True, help='save tensorboad log')
ap.add_argument('--cuda', action='store_true', default=False, help='')
ap.add_argument('--window', type=int, default=20, help='')
ap.add_argument('--horizon', type=int, default=5, help='leadtime default 5')
ap.add_argument('--gpu', type=int, default=0, help='choose gpu 0-10')
ap.add_argument('--lamda', type=float, default=0.01, help='regularize params similarities of states')
ap.add_argument('--patience', type=int, default=50, help='patience default 100')
ap.add_argument('--k', type=int, default=8, help='kernels')
ap.add_argument('--hidA', type=int, default=64, help='hidden dim of attention layer')
ap.add_argument('--hidP', type=int, default=1, help='hidden dim of adaptive pooling')
ap.add_argument('--extra', type=str, default='', help='externel folder')
ap.add_argument('--label', type=str, default='', help='label_file')
ap.add_argument('--pcc', type=str, default='', help='have pcc?')
ap.add_argument('--n', type=int, default=2, help='layer number of GCN')
ap.add_argument('--res', type=int, default=0, help='0 means no residual link while 1 means need residual link')
ap.add_argument('--s', type=int, default=2, help='kernel size of temporal convolution network')
ap.add_argument('--result', type=int, default=0, help='0 means do not show result while 1 means show result')
ap.add_argument('--ablation', type=str, default=None, help='ablation test')
ap.add_argument('--eval', type=str, default='', help='evaluation test file')
ap.add_argument('--record', type=str, default='', help='record the result')
ap.add_argument('--model', type=str, default='GAT', help='model')
args = ap.parse_args([])

args.runTag = -1  # -1说明是初始生成轮,该文件中必须为-1
args.global_nei_num = 0  # 邻居数量，该文件中必须为0
args.hidR = args.k * 4 * args.hidP + args.k #不能修改




In [34]:
#tasktype = 1
import os, itertools, random, argparse, time, datetime,torch
parser = argparse.ArgumentParser()
parser.add_argument('--tasktype', type=int, default='1', help="task type, 0: dynamics,1:node_level, 2:edge_level")
parser.add_argument('--dataset', default='computers', help='dataset name')
parser.add_argument('--net', type=str, default='GCN')
parser.add_argument('--RPMAX', type=int, default=10, help='repeat times')
parser.add_argument('--epochs', type=int, default=10)
parser.add_argument('--seed', type=int, default=42, help='random seed')
parser.add_argument('--clientNum', type=int, default=3)
parser.add_argument('--neighborAgencyNum', type=int, default=0)
parser.add_argument('--runTag', type=int, default=-1, help='local模型中固定为-1')

parser.add_argument('--early_stopping', type=int, default=200)
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--weight_decay', type=float, default=0.0001)
parser.add_argument('--dropout', type=float, default=0.3)
parser.add_argument('--train_rate', type=float, default=0.6)
parser.add_argument('--val_rate', type=float, default=0.2)
parser.add_argument('--hidden', type=int, default=32)
parser.add_argument('--print_freq', type=int, default=20)
parser.add_argument('--cuda', type=int, default=0)
args = parser.parse_args([])

In [24]:
#tasktype = 2
parser = argparse.ArgumentParser()
parser.add_argument('--tasktype', type=int, default='2', help="task type, 0: dynamics,1:node_level, 2:edge_level")
parser.add_argument('--seed', type=int, default=0,
                    help='ramdon seed')
parser.add_argument('--device', type=int, default=0,
                    help='')
parser.add_argument('--client_num_in_total', type=int, default=3,
                    help='')
parser.add_argument('--learning_rate', type=float, default=0.003,
                    help='')
parser.add_argument('--weight_decay', type=float, default=0.001,
                    help='')
parser.add_argument('--metric', type=str, default='MAE',
                    help='')
parser.add_argument('--epochs', type=int, default=1000,
                    help='')
parser.add_argument('--client_optimizer', type=str, default='sgd',
                    help='')
parser.add_argument('--model_name', type=str, default='gcn',
                    help='')
args = parser.parse_args([])




In [35]:
import zerorpc
import uuid
from SN1.security import *
##初始化
task_id = str(uuid.uuid4())
raw_task_id = task_id
print(task_id)
pu = loadPublicKey('./SN1/public.pem')
text = pickle.dumps(task_id)
task_id = encrypt(text, pu)

cstring = ('tcp://%s:%d'%('127.0.0.1',20001))
c = zerorpc.Client(heartbeat=None)
c.connect(cstring)
c.iniItask(task_id)
_,_,d = c.get_nei_embed(pickle.dumps(args), raw_task_id)
while True:
    ##等待计算完毕，每10秒检测1次
    a = pickle.loads(c.get_nei_embed_sum(raw_task_id, d))
    if not isinstance(a,int):
        res = a
        break
    time.sleep(5)
#邻居嵌入之和
res

29baef93-4ea9-4844-a783-a927ced2a1b9


tensor([[0.0019, 0.0025, 0.0015, 0.0008, 0.0035, 0.0018, 0.0041, 0.0036, 0.0029,
         0.0009, 0.0022, 0.0027, 0.0044, 0.0014, 0.0008, 0.0038, 0.0021, 0.0047,
         0.0041, 0.0018, 0.0038, 0.0033, 0.0047, 0.0032, 0.0020, 0.0020, 0.0010,
         0.0013, 0.0045, 0.0041, 0.0026, 0.0018, 0.0018, 0.0017, 0.0014, 0.0030,
         0.0017, 0.0024, 0.0019, 0.0025, 0.0011, 0.0030, 0.0013, 0.0021, 0.0017,
         0.0012, 0.0027, 0.0017, 0.0017, 0.0041, 0.0034, 0.0014, 0.0051, 0.0023,
         0.0049, 0.0044, 0.0018, 0.0015, 0.0019, 0.0014, 0.0015, 0.0049, 0.0023,
         0.0010, 0.0028, 0.0059, 0.0003, 0.0037, 0.0017, 0.0018, 0.0014, 0.0016,
         0.0029, 0.0056, 0.0051, 0.0030, 0.0020, 0.0015, 0.0049, 0.0006, 0.0017,
         0.0018, 0.0031, 0.0012, 0.0021, 0.0036, 0.0056, 0.0036, 0.0027, 0.0024,
         0.0065, 0.0034, 0.0028, 0.0019, 0.0020, 0.0020, 0.0027, 0.0042, 0.0021,
         0.0061, 0.0015, 0.0072, 0.0030, 0.0011, 0.0020, 0.0011, 0.0010, 0.0019,
         0.0012, 0.0024, 0.0